In [1]:
#%% Imports
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import skimage.io
from tqdm import tqdm

# Import "local" packages
import hyspec_io, preprocess, image_render

In [2]:
# Parameters
nir_band = (730,800)
Rrs_stretch_limits = np.array((0.003,0.012))

In [3]:
# Paths
image_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/Hyperspectral/20230621/2a_Rrs')
image_output_dir = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/Hyperspectral/20230621/3a_Rrs_FSSGC')
image_output_dir.mkdir(exist_ok=True)

image_paths = sorted(image_dir.glob('*.hdr'))

In [4]:
for image_path in tqdm(image_paths[10::20]):
    # Load image
    (image,wl,rgb_ind,metadata) = hyspec_io.load_envi_image(image_path)
    
    # Remove glint
    image_noglint = preprocess.remove_glint_flatspec(image,wl,nir_band=nir_band)

    # Save hyspec image
    image_save_path = image_output_dir / image_path.name
    hyspec_io.save_envi_image(image_save_path,image_noglint,metadata)

    # Create RGB version (percentile stretch)
    # image_rgb = image_render.percentile_stretch(image_noglint[:,:,rgb_ind])
    image_rgb = image_render.absolute_stretch(image_noglint[:,:,rgb_ind],Rrs_stretch_limits)
    image_rgb = np.uint8(image_rgb*255)  # Convert to 8-bit int
    transparency_layer = np.ones(shape=(image_rgb.shape[0:2])+(1,),dtype=np.uint8)*255
    image_rgb = np.concatenate((image_rgb,transparency_layer),axis=2)

    # Save RGB image
    hyspec_base_name = image_path.name.split('.')[0]
    rgg_image_save_path = image_output_dir / (hyspec_base_name + '.png')
    skimage.io.imsave(rgg_image_save_path,image_rgb)


100%|██████████| 6/6 [01:56<00:00, 19.50s/it]
